In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import glow.lenses as lenses
import glow.mismatch  as mm
import glow.waveform as wf
import glow.tools as tools

import glow.physical_units as pu
pu.initialize_cosmology()

from scipy.integrate import simps
import astropy.units as u

In [ ]:
# setting up the unlensed waveform

# # # # LISA 

detector='LISA'
z_src_waveform= 5
Mtot=1e6
Mtot_detector= Mtot*(1+z_src_waveform)
q=1
spin=0
inc=0
Tobs= 0.46*pu.yr_to_s
Mchirp=(q/(1+q))**(3/5)*Mtot

print('{:.1e}'.format(Mchirp))

params_source= {'approximant': "IMRPhenomXHM",
            'mass1'          : Mtot_detector * q/(1. + q),
            'mass2'          : Mtot_detector * 1/(1. + q),
            'spin1z'         : spin,
            'spin2z'         : spin,
            'redshift'       : z_src_waveform,
            'inclination'    : inc,
            'long_asc_nodes' : 0,
            'f_lower'        : np.amax([wf.f0_obs(Mtot_detector, Tobs, units='s'),wf.f_bounds_detector(detector)[0]]),
            'delta_f'        : 1/Tobs,
            'f_final':  7*wf.f_isco(Mtot_detector),
            }

# A waveform is generated once a waveform object is initialized by the source parameters 
h_fd=wf.WaveformFD(params_source)
psd=wf.get_psd_from_file(detector) 
h_fd.load_psd(psd)

snr=h_fd.snr

In [ ]:
# !! Check that the signal meets the psd on the lhs

plt.loglog(h_fd.sample_frequencies,h_fd.sample_frequencies*np.abs(h_fd.p))
plt.loglog(h_fd.sample_frequencies,np.sqrt(h_fd.sample_frequencies*np.abs(h_fd.psd_grid)))
plt.xlabel('$f$ [Hz]')
plt.ylabel('Characteristic strain')
plt.grid()
print('snr of the signal:', h_fd.snr)

### NFW 

Lens potential

In [ ]:
def get_Psis_NFW(Mvir_NFW_s, z_lens, z_src_waveform, norm_factor=1):

    # Calculate lensing masses (MLzs) for NFW profile
    MLz_NFW_s = [pu.to_MLz_NFW(Mvir, z_lens) for Mvir in Mvir_NFW_s]

    # Calculate the effective distance ratio (deff)
    deff = pu.get_deff(z_lens, z_src_waveform)

    # Calculate the Einstein radius (xi_0) for NFW profile and normalize it
    xi_0_NFW_s = [norm_factor * pu.get_R_E(MLz, deff) for MLz in MLz_NFW_s]

    # Calculate the scale radius (rs) for NFW profile
    rs_s = [pu.get_rs_NFW(Mvir, z_lens) for Mvir in Mvir_NFW_s]

    # Calculate the dimensionless scale parameter xs = rs / xi_0 and extract the values
    xs_s = [(rs / xi_0).decompose().value for (rs, xi_0) in zip(rs_s, xi_0_NFW_s)]

    norm_factor = [1 / norm_factor] * len(xs_s)

    # Compute potential

    Psi_NFW_s=[lenses.Psi_NFW(p_phys={'psi0':(norm_factor[i])**2,'xs':xs},
                            p_prec = {'eps_soft': 1e-15, 'eps_NFW': 0.01}) for i, xs in enumerate(xs_s)]

    return Psi_NFW_s, MLz_NFW_s

In [ ]:
low=1e2
high=1e11

# Generate an array of virial masses
Mvir_NFW_s = np.geomspace(low, high, 15)  # Using LISA values, adjust if needed
# Set the redshift of the lens as half of the source redshift
z_lens = z_src_waveform / 2

Psi_NFW_s, MLz_NFW_s = get_Psis_NFW(Mvir_NFW_s, z_lens, z_src_waveform, norm_factor=1)

Psi_SIS=lenses.Psi_SIS()

Lensing functions

In [ ]:
from glow import time_domain_c
y=1e-2

Cprec = {'si_drivContour_taumin_over_y2' : 1e-3}

p_prec_t = {'tmin' : 1e-7,\
          'tmax' : 1e8,\
          'Nt' : 50000,
          'C_prec' : Cprec,
          'parallel' : True}

I_NFW_s=[time_domain_c.It_SingleIntegral_C(Psi_NFW, y=y, p_prec=p_prec_t) for Psi_NFW in Psi_NFW_s]

taus=np.geomspace(p_prec_t['tmin'], p_prec_t['tmax'], num=1000)

fig, ax= plt.subplots(figsize=(7,4))
from matplotlib.cm import get_cmap
cmap = get_cmap('viridis')

l=0
h=-1
for i,(Mvir_NFW, I_NFW) in enumerate(zip(Mvir_NFW_s[l:h], I_NFW_s[l:h])):
    c=cmap(i/len(Mvir_NFW_s[l:h]))
    plt.semilogx(taus, I_NFW(taus), c=c, label='${:s}\,M_\odot$'.format(tools.latex_float(Mvir_NFW)))

plt.grid()
ax.set_ylabel('$I(\\tau)$')
ax.set_xlabel('$\\tau$')
# ax.set_xlim(1e-7,1e3)
# ax.set_ylim(0.995,1.001)
plt.legend(title='$M_{\\rm vir}$', edgecolor='white', loc='upper right')

In [ ]:

from glow import freq_domain_c


p_prec_f_opt_s=[]
p_prec_f={'N_below_discard':8, 'N_above_discard':8}

# Updates p_prec_f with waveform specific wmin, wmax
for MLz in MLz_NFW_s:
    p_prec_t_opt, p_prec_f_opt = wf.get_p_prec_optimal(h_fd.p, MLz) 
    for k,v in zip(p_prec_f.keys(), p_prec_f.values()):
        p_prec_f_opt[k]=v
    p_prec_f_opt_s.append(p_prec_f_opt)

# FFT
F_NFW_s=[freq_domain_c.Fw_FFT_C(I_NFW, p_prec_f_opt) for p_prec_f_opt, I_NFW in zip(p_prec_f_opt_s,I_NFW_s)]


fig, ax= plt.subplots(figsize=(7,4))
from matplotlib.cm import get_cmap
cmap = get_cmap('viridis')

l=0
h=-1
for i, (Mvir_NFW, p_prec_f_opt, F_NFW) in enumerate(zip(Mvir_NFW_s[l:h],p_prec_f_opt_s[l:h], F_NFW_s[l:h])):
    c=cmap(i/len(Mvir_NFW_s[l:h]))
    ws=np.geomspace(p_prec_f_opt['wmin'], p_prec_f_opt['wmax'], 100)
    plt.plot(ws, np.abs(F_NFW(ws)), c=c, alpha=0.7,
             label='${:s}\,M_\odot$'.format(tools.latex_float(Mvir_NFW)))

plt.grid()
ax.set_ylabel('$F(w)$')
ax.set_xlabel('$w$')
ax.set_xscale('log')
ax.set_yscale('linear')
plt.legend(title='$M_{\\rm vir}$', edgecolor='white', loc=2)

Critical curves

In [ ]:
ysl_s=[mm.find_ysl(Psi_NFW, xmin=1e-20, xmax=1e2, N=10000) for Psi_NFW in Psi_NFW_s]
print(ysl_s)

In [ ]:
y_crit_NFW_s=mm.get_y_crit_curve_opt(h_fd, Psi_NFW_s, MLz_NFW_s, 1e-3, 1, 1, 
                                    p_prec_t=p_prec_t, p_prec_f=p_prec_f, robust=False, SL=True)
print(y_crit_NFW_s)

In [ ]:
# Comparison with SIS
Psi_SIS=lenses.Psi_SIS()
MLz_SIS_s=pu.to_MLz_SIS(Mvir_NFW_s, z_src_waveform, z_lens)
deff = pu.get_deff(z_lens, z_src_waveform)
xi_0_SIS_s=[pu.get_R_E(MLz_SIS, deff) for MLz_SIS in MLz_SIS_s]
y_crit_SIS_s=mm.get_y_crit_curve_opt(h_fd, Psi_SIS, MLz_SIS_s, 1, 100, 1, p_prec_t={'tmax':1e9})


In [ ]:
y_crit_NFW_phys_s=[pu.to_phys_NFW(y_crit_NFW, Mvir_NFW, z_lens, z_src_waveform).value for (y_crit_NFW, Mvir_NFW) in zip(y_crit_NFW_s, Mvir_NFW_s)]
y_crit_SIS_phys_s=[pu.to_phys_SIS(y_crit_SIS, Mvir_NFW, z_lens, z_src_waveform).value for (y_crit_SIS, Mvir_NFW) in zip(y_crit_SIS_s, Mvir_NFW_s)]


In [ ]:
plt.semilogx(Mvir_NFW_s, y_crit_NFW_phys_s, label='NFW', marker='.')

plt.semilogx(Mvir_NFW_s, y_crit_SIS_phys_s, label='SIS', marker='.')

plt.yscale('log')

plt.ylabel('$y_{\\rm crit}\\times \\xi_0 \, [{\\rm Mpc}]$')
plt.xlabel('$M_{\\rm vir}\, [M_\odot]$')
plt.title(r"$M_{{\rm BBH}}= {:s}\,M_\odot\,(z_s={:s})$".format(tools.latex_float(Mtot), tools.latex_float(z_src_waveform)), fontsize=14)
plt.grid()
plt.legend()

### NFW + SIS

Lenses potential

In [ ]:
def get_Psis_NFW(Mvir_NFW_s, z_lens, z_src_waveform, norm_factor=1):


    # Set normalization factor
    norm_factor = 1

    low=1e3
    high=1e11


    # Generate an array of virial masses
    Mvir_NFW_s = np.geomspace(low, high, 10)  # Using LISA values, adjust if needed
    # Set the redshift of the lens as half of the source redshift
    z_lens = z_src_waveform / 2

    # Calculate lensing masses (MLzs) for NFW profile
    MLzs_NFW = [pu.to_MLz_NFW(Mvir, z_lens) for Mvir in Mvir_NFW_s]

    # Calculate the effective distance ratio (deff)
    deff = pu.get_deff(z_lens, z_src_waveform)

    # Calculate the Einstein radius (xi_0) for NFW profile and normalize it
    xi_0_NFW_s = [norm_factor * pu.get_R_E(MLz, deff) for MLz in MLzs_NFW]

    # Calculate the scale radius (rs) for NFW profile
    rs_s = [pu.get_rs_NFW(Mvir, z_lens) for Mvir in Mvir_NFW_s]

    # Calculate the dimensionless scale parameter xs = rs / xi_0 and extract the values
    xs_s = [(rs / xi_0).decompose().value for (rs, xi_0) in zip(rs_s, xi_0_NFW_s)]

    norm_factor = [1 / norm_factor] * len(xs_s)

    # Compute potential

    Psi_NFW_s=[lenses.Psi_NFW(p_phys={'psi0':(norm_factor[i])**2,'xs':xs},
                            p_prec = {'eps_soft': 1e-15, 'eps_NFW': 0.01}) for i, xs in enumerate(xs_s)]

    return Psi_NFW_s, MLzs_NFW

def get_Psis_SIS(m, Mvir_NFW_s, MLzs_NFW, z_lens, z_src_waveform):

    MLzs_SIS=[pu.to_MLz_SIS(m*Mvir_NFW, z_src_waveform, z_lens) for Mvir_NFW in Mvir_NFW_s]
    Mvir_SIS_s=[pu.to_Mvir_SIS(MLz_SIS, z_src_waveform, z_lens) for MLz_SIS in MLzs_SIS]

    # The normalization scale is NFW's
    Psi_SIS_s=[lenses.Psi_SIS(p_phys={'psi0': np.sqrt(MLz_SIS/MLz_NFW)}) for MLz_NFW, MLz_SIS in zip(MLzs_NFW, MLzs_SIS)]

    return Psi_SIS_s, MLzs_SIS

In [ ]:
m=0.1

low=1e2
high=1e11

Mvir_NFW_s = np.geomspace(low, high, 10)  # Using LISA values, adjust if needed
# Set the redshift of the lens as half of the source redshift
z_lens = z_src_waveform/2

Psi_NFW_s, MLzs_NFW=get_Psis_NFW(Mvir_NFW_s, z_lens, z_src_waveform)
Psi_SIS_s, MLzs_SIS=get_Psis_SIS(m, Mvir_NFW_s, MLzs_NFW, z_lens, z_src_waveform)

Psi_comp_s = [lenses.CombinedLens({'lenses':[Psi_NFW, Psi_SIS]}) for Psi_NFW, Psi_SIS in zip(Psi_NFW_s, Psi_SIS_s)]
for Psi_comp in Psi_comp_s: Psi_comp.isAxisym=True 
# Compute SL limit

ysl_comp_s=mm.find_ysl(Psi_comp_s)

Lensing functions

In [ ]:
from glow import time_domain_c
y=1e-2

Cprec = {'si_drivContour_taumin_over_y2' : 1e-3}

print('interpolation starts at tau=', (Cprec['si_drivContour_taumin_over_y2']*y**2))
p_prec_t = {'tmin' : 1e-7,\
          'tmax' : 1e9,\
          'Nt' : 5000,
          'C_prec' : Cprec,
          'parallel' : True}

I_comp_s=[time_domain_c.It_SingleIntegral_C(Psi_comp, y=y, p_prec=p_prec_t) for Psi_comp in Psi_comp_s]
I_NFW_s=[time_domain_c.It_SingleIntegral_C(Psi_NFW, y=y, p_prec=p_prec_t) for Psi_NFW in Psi_NFW_s]
I_SIS_s=[time_domain_c.It_SingleIntegral_C(Psi_SIS, y=y, p_prec=p_prec_t) for Psi_SIS in Psi_SIS_s]

taus=np.geomspace(p_prec_t['tmin'], p_prec_t['tmax'], num=10000)

fig, ax= plt.subplots(figsize=(7,4))
from matplotlib.cm import get_cmap
cmap = get_cmap('viridis')

l=6
h=8


for i,(Mvir_NFW, I_comp, I_NFW, I_SIS) in enumerate(zip(Mvir_NFW_s[l:h], I_comp_s[l:h], I_NFW_s[l:h], I_SIS_s[l:h])):
    c=cmap(i/len(Mvir_NFW_s[l:h]))
    ax.semilogx(taus, I_comp(taus), c=c, label='${:s}\,M_\odot$'.format(tools.latex_float(Mvir_NFW)))
    ax.semilogx(taus, I_NFW(taus), ":", c=c)
    ax.semilogx(taus, I_SIS(taus), "--", c=c)

ax2=ax.twinx()
ax2.plot([],[], "k:", label='NFW')
ax2.plot([],[], "k--", label='SIS')
ax2.legend( edgecolor='white',loc='center right')

ax.grid()
ax.set_ylabel('$I(\\tau)$')
ax.set_xlabel('$\\tau$')
# ax.set_xlim(1e-7,1e3)
# ax.set_ylim(0.995,1.001)
ax.legend(title='$M_{\\rm vir}$', edgecolor='white', loc='upper right')

In [ ]:
from glow import freq_domain_c


p_prec_f_opt_s=[]
p_prec_f={'N_below_discard':8, 'N_above_discard':8 }
# p_prec_f={}

for MLz in MLzs_NFW:
    p_prec_t_opt, p_prec_f_opt = wf.get_p_prec_optimal(h_fd.p, MLz) 
    for k,v in zip(p_prec_f.keys(), p_prec_f.values()):
        p_prec_f_opt[k]=v
    p_prec_f_opt_s.append(p_prec_f_opt)



F_comp_s=[freq_domain_c.Fw_FFT_C(I_comp, p_prec_f_opt) for p_prec_f_opt, I_comp in zip(p_prec_f_opt_s,I_comp_s)]
F_SIS_s=[freq_domain_c.Fw_FFT_C(I_SIS, p_prec_f_opt) for p_prec_f_opt, I_SIS in zip(p_prec_f_opt_s,I_SIS_s)]
F_NFW_s=[freq_domain_c.Fw_FFT_C(I_NFW, p_prec_f_opt) for p_prec_f_opt, I_NFW in zip(p_prec_f_opt_s,I_NFW_s)]

fig, ax= plt.subplots(figsize=(7,4))
from matplotlib.cm import get_cmap
cmap = get_cmap('viridis')

l=4
h=8
for i, (Mvir_NFW, p_prec_f_opt, F_comp,F_SIS,F_NFW) in enumerate(zip(Mvir_NFW_s[l:h],p_prec_f_opt_s[l:h], F_comp_s[l:h],F_SIS_s[l:h],F_NFW_s[l:h])):
    c=cmap(i/len(Mvir_NFW_s[l:h]))
    ws=np.geomspace(p_prec_f_opt['wmin'], p_prec_f_opt['wmax'], 10000)
    ax.plot(ws, F_comp(ws), c=c, label='${:s}\,M_\odot$'.format(tools.latex_float(Mvir_NFW)))
    ax.plot(ws, F_SIS(ws), '--', c=c)
    ax.plot(ws, F_NFW(ws), ':', c=c)


ax2=ax.twinx()
ax2.plot([],[], linestyle='dashdot', c='k', label='sum')
ax2.plot([],[], "k:", label='NFW')
ax2.plot([],[], "k--", label='SIS')
ax2.legend( edgecolor='white',loc='upper left')

ax.grid()
ax.set_ylabel('$F(w)$')
ax.set_xlabel('$w$')
ax.set_xscale('log')
ax.set_yscale('linear')
ax.legend(title='$M_{\\rm vir}$', edgecolor='white', loc='lower left')

Critical curves

In [ ]:
ysl_comp_s=[mm.find_ysl(Psi_comp) for Psi_comp in Psi_comp_s]


In [ ]:
ysl_comp_s=np.array([mm.find_ysl(Psi_comp) for Psi_comp in Psi_comp_s])
y_crit_comp_s=mm.get_y_crit_curve_opt(h_fd, Psi_comp_s, MLzs_NFW, 1.1*ysl_comp_s, 100*ysl_comp_s, 1,
                                      p_prec_t=p_prec_t, p_prec_f=p_prec_f, robust=False, SL=False)

y_crit_NFW_s=mm.get_y_crit_curve_opt(h_fd, Psi_NFW_s, MLzs_NFW, 0.001, 10, 1, p_prec_t=p_prec_t, p_prec_f=p_prec_f, robust=False)

ysl_SIS_s=np.array([mm.find_ysl(Psi_SIS) for Psi_SIS in Psi_SIS_s])
y_crit_SIS_s=mm.get_y_crit_curve_opt(h_fd, Psi_SIS_s, MLzs_NFW, 1.1*ysl_SIS_s, 1e2*ysl_SIS_s, 1, 
                                    p_prec_t=p_prec_t, p_prec_f=p_prec_f, optimized= False,robust=False, SL=False)

In [ ]:
y_crit_comp_phys_s=[pu.to_phys_NFW(y_crit_comp, Mvir_NFW, z_lens, z_src_waveform).value for (y_crit_comp, Mvir_NFW) in zip(y_crit_comp_s, Mvir_NFW_s)]
y_crit_NFW_phys_s=[pu.to_phys_NFW(y_crit_NFW, Mvir_NFW, z_lens, z_src_waveform).value for (y_crit_NFW, Mvir_NFW) in zip(y_crit_NFW_s, Mvir_NFW_s)]
y_crit_SIS_phys_s=[pu.to_phys_NFW(y_crit_SIS, Mvir_NFW, z_lens, z_src_waveform).value for (y_crit_SIS, Mvir_NFW) in zip(y_crit_SIS_s, Mvir_NFW_s)]


In [ ]:
plt.semilogx(Mvir_NFW_s, y_crit_NFW_phys_s, label='NFW', linestyle='-', linewidth=2.4, alpha=0.7)
plt.semilogx(Mvir_NFW_s, y_crit_SIS_phys_s, label='SIS (m={:.1e})'.format(m), alpha=0.7, linewidth=2.4)
plt.semilogx(Mvir_NFW_s, y_crit_comp_phys_s, label='NFW+SIS', linestyle='-', linewidth=2.4, alpha=0.7)


plt.yscale('log')

plt.ylabel('$y_{\\rm crit}\\times \\xi_0 \, [{\\rm Mpc}]$')
plt.xlabel('$M_{\\rm vir}\, [M_\odot]$')
plt.title(r"$M_{{\rm BBH}}= {:s}\,M_\odot\,(z_s={:s})$".format(tools.latex_float(Mtot), tools.latex_float(z_src_waveform)), fontsize=14)
plt.grid()
plt.legend()